In [8]:
import numpy as np
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pydantic as pd
from typing import Tuple, Dict

import sys
sys.path.append('../../../')
from tidy3d import Simulation, PML

class App:
    
    def __init__(self, sims : Dict[str, Simulation]):
        self.sims = sims
        self.task_name = list(self.sims.keys())[0]
        self.app = self.make_app()
        
    def make_app(self):

        app = JupyterDash(__name__)

        app.layout = html.Div([
            dcc.Tabs([
                self.render_layout(app, task_name, sim)
                for task_name, sim in self.sims.items()])
        ])
                              
        return app


    def render_layout(self, app, task_name, sim):
        
        pml_labels = ['PML (x)', 'PML (y)', 'PML (z)']
        initial_labels = [label for label, pml_on in zip(pml_labels, sim.pml_layers) if pml_on is not None]
        
        def append_task_name(string):
            return f'{string}_{task_name}'
        
        layout = dcc.Tab([
            html.H2(f'Task: {task_name}'),

            # size
            html.Br(),
            html.Label('Size [x,y,z] (um)'),
            html.Br(),            
            html.Div([
                dcc.Input(value=sim.size[0], type='number', id=append_task_name('size_x'))
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([

                dcc.Input(value=sim.size[1], type='number', id=append_task_name('size_y'))
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([
                dcc.Input(value=sim.size[2], type='number', id=append_task_name('size_z'))
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Br(),

            
            # grid size
            html.Br(),
            html.Label('Grid Size [x,y,z] (um)'),
            html.Br(),            
            html.Div([
                dcc.Input(value=sim.grid_size[0], type='number', id=append_task_name('grid_size_x'))
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([

                dcc.Input(value=sim.grid_size[1], type='number', id=append_task_name('grid_size_y'))
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([
                dcc.Input(value=sim.grid_size[2], type='number', id=append_task_name('grid_size_z'))
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Br(),

            
            # run time
            html.Br(),
            html.Label('Run Time (ps)'),
            html.Br(),
            dcc.Input(value=sim.run_time*1e12, type='number', id=append_task_name('run_time')),
            html.Br(),


            # pml
            html.Br(),
            html.Label('PML Layers [x,y,z]'),
            html.Br(),
            dcc.Checklist(
                options=pml_labels,
                value=initial_labels,
                id=append_task_name('pml_checklist')
            ),
            html.Br(),


            # symmetry
            html.Br(),
            html.Label('Symmetry [x,y,z]'),
            html.Br(),            
            html.Div([
                dcc.Dropdown([-1, 0, 1], value=sim.symmetry[0], id=append_task_name('symmetry_x'))
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([
                dcc.Dropdown([-1, 0, 1], value=sim.symmetry[1], id=append_task_name('symmetry_y'))
            ], style={'width': '15%', 'display': 'inline-block'}),
            html.Div([
                dcc.Dropdown([-1, 0, 1], value=sim.symmetry[2], id=append_task_name('symmetry_z'))
            ], style={'width': '15%', 'display': 'inline-block'}),
    
            # subpixel
            html.Br(),
            html.Label('Subpixel Smoothening'),
            html.Br(),
            dcc.Checklist(
                options=['On',],
                value=['On'] if sim.subpixel else [],
                id=append_task_name('subpixel')
            ),
            html.Br(),

            # shutoff
            html.Br(),
            html.Label('Field Decay Shutoff'),
            html.Br(),
            dcc.Input(value=sim.shutoff, type='number', id=append_task_name('shutoff')),   
            html.Br(),

            
            # courant
            html.Br(),
            html.Label('Courant Number'),
            html.Br(),
            dcc.Input(value=sim.courant, type='number', id=append_task_name('courant')),   
            html.Br(),
        ],
        label=f'task: {task_name}',
        )
        
        @app.callback(
            Output(append_task_name('pml_checklist'), 'value'),
            Input(append_task_name('pml_checklist'), 'value')
        )
        def set_model(value):
            pml_labels = ['PML (x)', 'PML (y)', 'PML (z)']            
            pml_on_list = [label in value for label in pml_labels]
            self.sims[self.task_name].pml_layers = [PML() if pml_on else None for pml_on in pml_on_list]
            # print([True if pml_on else None for pml_on in pml_on_list])
            print(f'sim.pml_layers set to {["PML" if pml else "None" for pml in pml_on_list]}')
            return value

        @app.callback(
            Output(append_task_name('run_time'), 'value'),
            Input(append_task_name('run_time'), 'value')
        )
        def set_run_time(value):
            new_value = max(0, value)
            new_value = value

            self.sims[self.task_name].run_time = new_value * 1e-12
            print(f'sim.run_time set to {new_value:.2f} ps')
            return new_value

        return layout
            
    def run(self):
    
        self.app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [9]:
sim1 = Simulation(size=(1,1,1), grid_size=(.01, .01, .01), run_time=1e-12, pml_layers=(PML(), PML(), None))
sim2 = Simulation(size=(2,3,4), grid_size=(.01, .01, .01), run_time=4e-12, pml_layers=(None, PML(), None))

app = App({'sim1': sim1, 'sim2': sim2})

[14:27:44] WARNING  No sources in simulation.                               ]8;id=916518;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/components/simulation.py\simulation.py]8;;\:]8;id=23300;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/components/simulation.py#366\366]8;;\

           WARNING  No sources in simulation.                               ]8;id=490174;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/components/simulation.py\simulation.py]8;;\:]8;id=822979;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/components/simulation.py#366\366]8;;\

In [10]:
app.run()

sim.run_time set to 1.00 ps
sim.pml_layers set to ['PML', 'PML', 'PML']
sim.run_time set to 4.00 ps
sim.pml_layers set to ['PML', 'PML', 'PML']
sim.run_time set to 2.00 ps
sim.run_time set to 3.00 ps


In [4]:
app.sims[0]

sim.run_time set to 1.00 pssim.pml_layers set to ['PML', 'PML', 'PML']
sim.run_time set to 4.00 ps

sim.pml_layers set to ['PML', 'PML', 'PML']


KeyError: 0